In [50]:
import pandas as pd
from nltk.tokenize import TweetTokenizer
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm

In [93]:
stopwords = ["a", "about", "after", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been",
            "before", "being", "between", "both", "by", "could", "did", "do", "does", "doing", "during", "each",
            "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here",
            "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've",
            "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "of",
            "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "own", "shan't", "she", "she'd",
            "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs",
            "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're",
            "they've", "this", "those", "through", "to", "until", "up", "very", "was", "wasn't", "we", "we'd",
            "we'll", "we're", "we've", "were", "weren't", "what", "what's", "when", "when's", "where", "where's",
            "which", "while", "who", "who's", "whom", "with", "would", "you", "you'd", "you'll", "you're", "you've",
            "your", "yours", "yourself", "yourselves", "above", "again", "against", "aren't", "below", "but", "can't",
            "cannot", "couldn't", "didn't", "doesn't", "don't", "down", "few", "hadn't", "hasn't", "haven't", "if",
            "isn't", "mustn't", "no", "nor", "not", "off", "out", "over", "shouldn't", "same", "too", "under", "why",
            "why's", "won't", "wouldn't", ".", ";", ",", ":", "?", "!", "\"", "-", "'", "...",
            "1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "...", "&"]

In [206]:
from sklearn.cross_validation import train_test_split

tweetfile = pd.read_csv('train.csv', sep=',', na_values=['Not Available']).dropna()
le_category = preprocessing.LabelEncoder()
# tweetfile.Category = le_category.fit_transform(tweetfile.Category)

# print(set(tweetfile.Category))
train_data, test_data = train_test_split(tweetfile, test_size=0.1, random_state=42)

traincorpus = list(train_data.Tweet)#[0:500]
traincategories = list(train_data.Category)#[0:500]
testcorpus = list(test_data.Tweet)
testcategories = list(test_data.Category)

tknzr = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=False)
vectorizer = TfidfVectorizer(min_df=3, strip_accents='ascii', stop_words=stopwords, use_idf=True, tokenizer=tknzr.tokenize)
Xtrain = vectorizer.fit_transform(traincorpus)
Xtest = vectorizer.transform(testcorpus)

In [266]:
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.metrics.classification import classification_report

classifier = GradientBoostingClassifier(random_state=42, n_estimators=1000, verbose=True)
svcfit = classifier.fit(Xtrain.toarray(), traincategories)

      Iter       Train Loss   Remaining Time 


         1        5993.8412           28.00m


         2        5830.6996           30.93m


         3        5690.2292           31.83m


         4        5567.8600           32.32m


...

       600        2447.3679            9.91m


       700        2285.8965            7.35m


       800        2137.9564            4.85m


       900        2004.5086            2.41m


      1000        1881.7078            0.00s


In [267]:
svcpred = classifier.predict(Xtest.toarray())

print(classification_report(testcategories, svcpred))

             precision    recall  f1-score   support

   negative       0.35      0.23      0.28        84
    neutral       0.48      0.47      0.48       199
   positive       0.64      0.73      0.68       260

avg / total       0.54      0.56      0.55       543



In [241]:
submissionfile = pd.read_csv('test.csv', dtype={'Id': object}, sep=',')
# print(submissionfile)
submissioncorpus = list(submissionfile.Category) # bad column name, that's actually tweets
Xsubm = vectorizer.transform(submissioncorpus)
svcpred_subm = classifier.predict(Xsubm.toarray())
submissionfile.Category = svcpred_subm
submissionfile.to_csv('subm_grad.csv', index=False)